<a href="https://colab.research.google.com/github/wlazlod/Elections/blob/master/Wybory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jak wyglądałby rozkład mandatów do Sejmu, gdyby zastosowano okręgi do Senatu (100 okręgów zamiast 41)

In [1]:
import pandas as pd
import numpy as np

## Pobranie danych i ich rozpakowanie.

Dane dotyczące wyborów parlamentarnych z 2019 roku znajdują się tutaj:
https://sejmsenat2019.pkw.gov.pl/sejmsenat2019/pl/dane_w_arkuszach

### Wyniki głosowania na listy wyborcze do sejmu w obwodach

Te wyniki (na najniższym poziomie granularności) zostaną wykorzystane jako źródło danych do zasymulowania wyborów.

Dane będą przechowywane w DataFrame `sejm_voting_results_2019`

In [2]:
!wget https://sejmsenat2019.pkw.gov.pl/sejmsenat2019/data/csv/wyniki_gl_na_listy_po_obwodach_sejm_xlsx.zip

--2020-07-26 18:37:49--  https://sejmsenat2019.pkw.gov.pl/sejmsenat2019/data/csv/wyniki_gl_na_listy_po_obwodach_sejm_xlsx.zip
Resolving sejmsenat2019.pkw.gov.pl (sejmsenat2019.pkw.gov.pl)... 193.219.114.9
Connecting to sejmsenat2019.pkw.gov.pl (sejmsenat2019.pkw.gov.pl)|193.219.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4716518 (4.5M) [application/zip]
Saving to: ‘wyniki_gl_na_listy_po_obwodach_sejm_xlsx.zip’

wyniki_gl_na_listy_ 100%[===================>]   4.50M  1.53MB/s    in 2.9s    

2020-07-26 18:37:53 (1.53 MB/s) - ‘wyniki_gl_na_listy_po_obwodach_sejm_xlsx.zip’ saved [4716518/4716518]



In [3]:
!unzip wyniki_gl_na_listy_po_obwodach_sejm_xlsx.zip

Archive:  wyniki_gl_na_listy_po_obwodach_sejm_xlsx.zip
  inflating: wyniki_gl_na_listy_po_obwodach_sejm.xlsx  


In [4]:
sejm_voting_results_2019 = pd.read_excel("wyniki_gl_na_listy_po_obwodach_sejm.xlsx",
                                         na_values = '-')

In [5]:
sejm_voting_results_2019.head()

,Symbol kontrolny,Kod TERYT,Okręg,Numer,Typ obszaru,Typ obwodu,Siedziba,Gmina,Powiat,Województwo,Komisja otrzymała kart do głosowania,Liczba wyborców uprawnionych do głosowania,Nie wykorzystano kart do głosowania,"Liczba wyborców, którym wydano karty do głosowania",Liczba wyborców głosujących przez pełnomocnika,Liczba wyborców głosujących na podstawie zaświadczenia o prawie do głosowania,"Liczba wyborców, którym wysłano pakiety wyborcze",Liczba otrzymanych kopert zwrotnych,"Liczba kopert zwrotnych, w których nie było oświadczenia o osobistym i tajnym oddaniu głosu","Liczba kopert zwrotnych, w których oświadczenie nie było podpisane","Liczba kopert zwrotnych, w których nie było koperty na kartę do głosowania","Liczba kopert zwrotnych, w których znajdowała się niezaklejona koperta na kartę do głosowania",Liczba kopert na kartę do głosowania wrzuconych do urny,Liczba kart wyjętych z urny,W tym liczba kart wyjętych z kopert na kartę do głosowania,Liczba kart nieważnych,Liczba kart ważnych,Liczba głosów nieważnych,W tym z powodu postawienia znaku „X” obok nazwiska dwóch lub większej liczby kandydatów z różnych list,W tym z powodu niepostawienia znaku „X” obok nazwiska żadnego kandydata,"W tym z powodu postawienia znaku „X” wyłącznie obok nazwiska kandydata na liście, której rejestracja została unieważniona",Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY AKCJA ZAWIEDZIONYCH EMERYTÓW RENCISTÓW - ZPOW-601-21/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWICA - ZPOW-601-20/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SKUTECZNI PIOTRA LIROYA-MARCA - ZPOW-601-17/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW KOALICJA BEZPARTYJNI I SAMORZĄDOWCY - ZPOW-601-10/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19
0,6a18-a07e-c240-e60e-2b2c-d9ee-4b6f-30b6,20101.0,1,1,miasto,stały,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...",m. Bolesławiec,bolesławiecki,dolnośląskie,1400.0,1522.0,512.0,888.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,888.0,0.0,0.0,888.0,6.0,1.0,5.0,0.0,882.0,202.0,NaN,72.0,46.0,NaN,368.0,NaN,145.0,49.0,NaN
1,87e0-d41a-ce4f-cde5-c934-67c7-4577-0bdd,20101.0,1,2,miasto,stały,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...",m. Bolesławiec,bolesławiecki,dolnośląskie,1200.0,1338.0,323.0,877.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,877.0,0.0,0.0,877.0,7.0,5.0,2.0,0.0,870.0,223.0,NaN,57.0,41.0,NaN,307.0,NaN,196.0,46.0,NaN
2,ec4f-7f57-b0eb-d14d-85b3-0095-8b38-265c,20101.0,1,3,miasto,stały,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...",m. Bolesławiec,bolesławiecki,dolnośląskie,1300.0,1432.0,397.0,903.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,903.0,0.0,0.0,903.0,10.0,7.0,3.0,0.0,893.0,193.0,NaN,55.0,52.0,NaN,331.0,NaN,194.0,68.0,NaN
3,5440-a0cd-5860-fdab-0a77-7c24-e35d-da10,20101.0,1,4,miasto,stały,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...",m. Bolesławiec,bolesławiecki,dolnośląskie,1300.0,1439.0,347.0,953.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,951.0,0.0,0.0,951.0,6.0,4.0,2.0,0.0,945.0,234.0,NaN,65.0,53.0,NaN,355.0,NaN,191.0,47.0,NaN
4,1ff9-1ec2-d664-fc91-5eee-71b7-d300-904e,20101.0,1,5,miasto,stały,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...",m. Bolesławiec,bolesławiecki,dolnośląskie,1300.0,1384.0,469.0,831.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,831.0,0.0,0.0,831.0,12.0,5.0,7.0,0.0,819.0,215.0,NaN,33.0,54.0,NaN,288.0,NaN,176.0,53.0,NaN


### Dane dotyczące obwodów głosowania

Dane te zostaną wykorzystane do przypisania poszczególnych obwodów do okręgów głosowania do Senatu (aby odtworzyć podział na okręgi w Krakowie, Łodzi i Wrocławiu). 

In [6]:
!wget https://sejmsenat2019.pkw.gov.pl/sejmsenat2019/data/csv/obwody_glosowania_xlsx.zip

--2020-07-26 18:38:24--  https://sejmsenat2019.pkw.gov.pl/sejmsenat2019/data/csv/obwody_glosowania_xlsx.zip
Resolving sejmsenat2019.pkw.gov.pl (sejmsenat2019.pkw.gov.pl)... 193.219.114.9
Connecting to sejmsenat2019.pkw.gov.pl (sejmsenat2019.pkw.gov.pl)|193.219.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4013103 (3.8M) [application/zip]
Saving to: ‘obwody_glosowania_xlsx.zip’

obwody_glosowania_x 100%[===================>]   3.83M  1.47MB/s    in 2.6s    

2020-07-26 18:38:27 (1.47 MB/s) - ‘obwody_glosowania_xlsx.zip’ saved [4013103/4013103]



In [7]:
!unzip obwody_glosowania_xlsx.zip

Archive:  obwody_glosowania_xlsx.zip
  inflating: obwody_glosowania.xlsx  


In [8]:
voting_districts = pd.read_excel("obwody_glosowania.xlsx",
                                 na_values = '-')

In [9]:
voting_districts.head()

,TERYT gminy,Gmina,Powiat,Numer,Mieszkańcy,Wyborcy,Siedziba,Miejscowość,Ulica,Numer posesji,Numer lokalu,Kod pocztowy,Poczta,Typ obwodu,Przystosowany dla niepełnosprawnych,Typ obszaru,Pełna siedziba,Opis granic,Numer okręgu do Sejmu,Numer okręgu do Senatu
0,20101,m. Bolesławiec,bolesławiecki,1,1914,1520,Szkoła Podstawowa Nr 3,Bolesławiec,ul. Ceramiczna,5,NaN,59-700,Bolesławiec,stały,Nie,miasto,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...","Agatowa, Bazaltowa, Bukowa, Ceramiczna 1 - 13 ...",1,1
1,20101,m. Bolesławiec,bolesławiecki,2,1646,1335,Szkoła Podstawowa Nr 3,Bolesławiec,ul. Ceramiczna,5,NaN,59-700,Bolesławiec,stały,Nie,miasto,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...","Astrów, Bławatkowa, Bratków, Czerwonych Maków,...",1,1
2,20101,m. Bolesławiec,bolesławiecki,3,1786,1428,Szkoła Podstawowa Nr 5,Bolesławiec,ul. Dolne Młyny,60,NaN,59-700,Bolesławiec,stały,Tak,miasto,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...","Ceramiczna parzyste 14 - 24a i od 25 do końca,...",1,1
3,20101,m. Bolesławiec,bolesławiecki,4,1820,1431,Szkoła Podstawowa Nr 5,Bolesławiec,ul. Dolne Młyny,60,NaN,59-700,Bolesławiec,stały,Tak,miasto,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...","Akacjowa, Brzozowa, Elizy Orzeszkowej, Góralsk...",1,1
4,20101,m. Bolesławiec,bolesławiecki,5,1714,1377,Szkoła Podstawowa Nr 5,Bolesławiec,ul. Dolne Młyny,60,NaN,59-700,Bolesławiec,stały,Tak,miasto,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...","Augusta Cieszkowskiego, Dębowa, Franciszka Sal...",1,1


### Dane dotyczące okręgów do Senatu (tylko po to by dopisać do wyników ich zasięg terytorialny)

In [62]:
!wget https://sejmsenat2019.pkw.gov.pl/sejmsenat2019/data/csv/okregi_senat_xlsx.zip

--2020-07-26 20:01:07--  https://sejmsenat2019.pkw.gov.pl/sejmsenat2019/data/csv/okregi_senat_xlsx.zip
Resolving sejmsenat2019.pkw.gov.pl (sejmsenat2019.pkw.gov.pl)... 193.219.114.9
Connecting to sejmsenat2019.pkw.gov.pl (sejmsenat2019.pkw.gov.pl)|193.219.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12228 (12K) [application/zip]
Saving to: ‘okregi_senat_xlsx.zip’

okregi_senat_xlsx.z 100%[===================>]  11.94K  --.-KB/s    in 0.001s  

2020-07-26 20:01:08 (14.0 MB/s) - ‘okregi_senat_xlsx.zip’ saved [12228/12228]



In [63]:
!unzip okregi_senat_xlsx.zip

Archive:  okregi_senat_xlsx.zip
  inflating: okregi_senat.xlsx       


In [64]:
senat_districts_teritory = pd.read_excel("okregi_senat.xlsx",
                                 na_values = '-')

In [65]:
senat_districts_teritory

,Siedziba OKW,Numer okręgu,Liczba kandydatów,Mieszkańcy,Wyborcy,Opis granic
0,Legnica,1,3,268968,219010,"obszary powiatów: bolesławiecki, lubański, lwó..."
1,Legnica,2,3,266939,219472,"obejmujący obszary powiatów: jaworski, jelenio..."
2,Legnica,3,3,391543,313689,"obszary powiatów: głogowski, legnicki, lubińsk..."
3,Wałbrzych,4,3,303901,250787,"obszary powiatów: świdnicki, wałbrzyski oraz m..."
4,Wałbrzych,5,3,312480,258392,"obszary powiatów: dzierżoniowski, kłodzki, ząb..."
...,...,...,...,...,...,...
95,Kalisz,96,2,309497,248880,"obszary powiatów: jarociński, kaliski, pleszew..."
96,Szczecin,97,2,433155,352634,obszar powiatu: policki oraz miasta na prawach...
97,Szczecin,98,2,548117,444089,"obszary powiatów: goleniowski, gryficki, gryfi..."
98,Koszalin,99,3,318648,258025,"obszary powiatów: białogardzki, choszczeński, ..."


# Wyznaczenie przybliżonej liczby posłów przypadającej na każdy z okręgów

In [10]:
voting_districts_grouped = voting_districts.groupby(["Numer okręgu do Senatu"])["Mieszkańcy", "Wyborcy"].sum()
voting_districts_grouped = voting_districts_grouped.reset_index()
voting_districts_grouped

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Numer okręgu do Senatu,Mieszkańcy,Wyborcy
0,1,268968,219010
1,2,266939,219472
2,3,391543,313689
3,4,303901,250787
4,5,312480,258392
...,...,...,...
95,96,309497,248880
96,97,433155,352634
97,98,548117,444089
98,99,318648,258025


Zakładam, że liczba posłów przynależna do danego okręgu zależy od liczby mieszkańców. Zakładam także, że liczba posłów (460) się nie zmienia.

Wyznaczenie liczby posłów w danym okręgu przeprowadzam wg następującego algorytmu:
1. Obliczam całkowitą liczbę mieszkańców
2. Liczbę mieszkańców w danym okręgu dzielę przez całkowitą liczbę mieszkańców oraz mnożę przez liczbę posłów (460)
3. Wyznaczam częśc całkowitą dla każdego z okręgów.
4. Odejmuję sumę części całkowitych od liczby mandatów do obsadzenia (aby ustalić ile "ułamkowych" mandatów pozostało do obsadzenia)
5. Odejmuję dla każdego z okręgów część całkowitą z punktu 3. od ilorazu z punktu 2. 
6. Szereguję okręgi wg. "części ułamkowej" z punktu 5.
7. Wybieram okręgi o najwyższej "części ułamkowej" aby obsadzić brakujące do 460 mandaty.

In [11]:
#Punkt 1
inhabitants_total_number = voting_districts_grouped["Mieszkańcy"].sum()
mp_number = 460
print("Liczba mieszkańców:", inhabitants_total_number)
#Punkt 2:
voting_districts_grouped["Udział w populacji"] = voting_districts_grouped["Mieszkańcy"] / inhabitants_total_number * 460
#Punkt 3:
voting_districts_grouped["Część całkowita"] = voting_districts_grouped["Udział w populacji"].astype(int)
#Punkt 4:
seats_number_taken = voting_districts_grouped["Część całkowita"].sum()
seats_number_left = mp_number - seats_number_taken
print("Pozostała liczba mandatów do obsadzenia:", seats_number_left)
#Punkt 5:
voting_districts_grouped["Część ułamkowa"] = voting_districts_grouped["Udział w populacji"] - voting_districts_grouped["Część całkowita"]
#Punkt 6:
voting_districts_grouped = voting_districts_grouped.sort_values(by=['Część ułamkowa'], ascending = False)\
                           .reset_index(drop = True)\
                           .reset_index()\
                           .rename(columns={"index": "Kolejnosć"})
#Punkt 7:
voting_districts_grouped["Dotatkowy mandat"] = np.where(voting_districts_grouped['Kolejnosć']<seats_number_left, 1, 0)
voting_districts_grouped["Liczba mandatów"] = voting_districts_grouped["Część całkowita"] + voting_districts_grouped["Dotatkowy mandat"]
voting_districts_grouped = voting_districts_grouped.sort_values(by=['Numer okręgu do Senatu'])
voting_districts_grouped


Liczba mieszkańców: 36910074
Pozostała liczba mandatów do obsadzenia: 56


,Kolejnosć,Numer okręgu do Senatu,Mieszkańcy,Wyborcy,Udział w populacji,Część całkowita,Część ułamkowa,Dotatkowy mandat,Liczba mandatów
68,68,1,268968,219010,3.352073,3,0.352073,0,3
72,72,2,266939,219472,3.326787,3,0.326787,0,3
17,17,3,391543,313689,4.879692,4,0.879692,1,5
30,30,4,303901,250787,3.787434,3,0.787434,1,4
14,14,5,312480,258392,3.894351,3,0.894351,1,4
...,...,...,...,...,...,...,...,...,...
21,21,96,309497,248880,3.857175,3,0.857175,1,4
64,64,97,433155,352634,5.398290,5,0.398290,0,5
24,24,98,548117,444089,6.831030,6,0.831030,1,7
3,3,99,318648,258025,3.971222,3,0.971222,1,4


Sprawdzenie, czy liczba parlamentarzystów się zgadza

In [12]:
voting_districts_grouped["Liczba mandatów"].sum()

460

### Parę statystyk:

Największa liczba mandatów:

In [13]:
voting_districts_grouped[voting_districts_grouped["Liczba mandatów"] == voting_districts_grouped["Liczba mandatów"].max()]

,Kolejnosć,Numer okręgu do Senatu,Mieszkańcy,Wyborcy,Udział w populacji,Część całkowita,Część ułamkowa,Dotatkowy mandat,Liczba mandatów
42,42,6,618465,491960,7.707758,7,0.707758,1,8
4,4,30,639580,506754,7.970908,7,0.970908,1,8


Okręgi:
*   6 - powiaty: górowski, milicki, oleśnicki, oławski, strzeliński, średzki, trzebnicki, wołowski, wrocławski
*   30 - powiaty:	chrzanowski, myślenicki, oświęcimski, suski, wadowicki


Najmniejsza liczba mandatów:

In [14]:
voting_districts_grouped[voting_districts_grouped["Liczba mandatów"] == voting_districts_grouped["Liczba mandatów"].min()]

,Kolejnosć,Numer okręgu do Senatu,Mieszkańcy,Wyborcy,Udział w populacji,Część całkowita,Część ułamkowa,Dotatkowy mandat,Liczba mandatów
59,59,61,200477,162951,2.498489,2,0.498489,0,2


Okręg 61 - powiaty: bielski, hajnowski, siemiatycki, wysokomazowiecki

Dla porównania:
 - obecnie najwięcej jest w okręgu numer 19 (Warszawa) - 20 mandatów
 - obecnie najmniej jest w okręgu numer 28 (Częstochowa) - 7 mandatów

# Przypisanie okręgów z senatu do wyników wyborów w obwodach w Sejmie

In [15]:
voting_districts_selected = voting_districts[["TERYT gminy", "Numer", "Numer okręgu do Senatu"]]


In [16]:
sejm_voting_results_2019_new = pd.merge(sejm_voting_results_2019, 
                                        voting_districts_selected,
                                        how = 'left',
                                        left_on = ["Kod TERYT", "Numer"],
                                        right_on = ["TERYT gminy", "Numer"])
sejm_voting_results_2019_new

,Symbol kontrolny,Kod TERYT,Okręg,Numer,Typ obszaru,Typ obwodu,Siedziba,Gmina,Powiat,Województwo,Komisja otrzymała kart do głosowania,Liczba wyborców uprawnionych do głosowania,Nie wykorzystano kart do głosowania,"Liczba wyborców, którym wydano karty do głosowania",Liczba wyborców głosujących przez pełnomocnika,Liczba wyborców głosujących na podstawie zaświadczenia o prawie do głosowania,"Liczba wyborców, którym wysłano pakiety wyborcze",Liczba otrzymanych kopert zwrotnych,"Liczba kopert zwrotnych, w których nie było oświadczenia o osobistym i tajnym oddaniu głosu","Liczba kopert zwrotnych, w których oświadczenie nie było podpisane","Liczba kopert zwrotnych, w których nie było koperty na kartę do głosowania","Liczba kopert zwrotnych, w których znajdowała się niezaklejona koperta na kartę do głosowania",Liczba kopert na kartę do głosowania wrzuconych do urny,Liczba kart wyjętych z urny,W tym liczba kart wyjętych z kopert na kartę do głosowania,Liczba kart nieważnych,Liczba kart ważnych,Liczba głosów nieważnych,W tym z powodu postawienia znaku „X” obok nazwiska dwóch lub większej liczby kandydatów z różnych list,W tym z powodu niepostawienia znaku „X” obok nazwiska żadnego kandydata,"W tym z powodu postawienia znaku „X” wyłącznie obok nazwiska kandydata na liście, której rejestracja została unieważniona",Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY AKCJA ZAWIEDZIONYCH EMERYTÓW RENCISTÓW - ZPOW-601-21/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWICA - ZPOW-601-20/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SKUTECZNI PIOTRA LIROYA-MARCA - ZPOW-601-17/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW KOALICJA BEZPARTYJNI I SAMORZĄDOWCY - ZPOW-601-10/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19,TERYT gminy,Numer okręgu do Senatu
0,6a18-a07e-c240-e60e-2b2c-d9ee-4b6f-30b6,20101.0,1,1,miasto,stały,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...",m. Bolesławiec,bolesławiecki,dolnośląskie,1400.0,1522.0,512.0,888.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,888.0,0.0,0.0,888.0,6.0,1.0,5.0,0.0,882.0,202.0,NaN,72.0,46.0,NaN,368.0,NaN,145.0,49.0,NaN,20101.0,1.0
1,87e0-d41a-ce4f-cde5-c934-67c7-4577-0bdd,20101.0,1,2,miasto,stały,"Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-7...",m. Bolesławiec,bolesławiecki,dolnośląskie,1200.0,1338.0,323.0,877.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,877.0,0.0,0.0,877.0,7.0,5.0,2.0,0.0,870.0,223.0,NaN,57.0,41.0,NaN,307.0,NaN,196.0,46.0,NaN,20101.0,1.0
2,ec4f-7f57-b0eb-d14d-85b3-0095-8b38-265c,20101.0,1,3,miasto,stały,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...",m. Bolesławiec,bolesławiecki,dolnośląskie,1300.0,1432.0,397.0,903.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,903.0,0.0,0.0,903.0,10.0,7.0,3.0,0.0,893.0,193.0,NaN,55.0,52.0,NaN,331.0,NaN,194.0,68.0,NaN,20101.0,1.0
3,5440-a0cd-5860-fdab-0a77-7c24-e35d-da10,20101.0,1,4,miasto,stały,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...",m. Bolesławiec,bolesławiecki,dolnośląskie,1300.0,1439.0,347.0,953.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,951.0,0.0,0.0,951.0,6.0,4.0,2.0,0.0,945.0,234.0,NaN,65.0,53.0,NaN,355.0,NaN,191.0,47.0,NaN,20101.0,1.0
4,1ff9-1ec2-d664-fc91-5eee-71b7-d300-904e,20101.0,1,5,miasto,stały,"Szkoła Podstawowa Nr 5, ul. Dolne Młyny 60, 59...",m. Bolesławiec,bolesławiecki,dolnośląskie,1300.0,1384.0,469.0,831.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,831.0,0.0,0.0,831.0,12.0,5.0,7.0,0.0,819.0,215.0,NaN,33.0,54.0,NaN,288.0,NaN,176.0,53.0,NaN,20101.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27410,ca04-d048-8fcf-7d31-cd72-0f6d-46d9-958a,NaN,19,119,zagranica,zagranica,"Novo Selo, Camp Maréshal de 

Przypisanie obwodom z zagranicy okręgu 44

In [17]:
sejm_voting_results_2019_new["Numer okręgu do Senatu"] = sejm_voting_results_2019_new["Numer okręgu do Senatu"].fillna(44).astype(int)

In [18]:
sejm_voting_results_2019_new.tail()

,Symbol kontrolny,Kod TERYT,Okręg,Numer,Typ obszaru,Typ obwodu,Siedziba,Gmina,Powiat,Województwo,Komisja otrzymała kart do głosowania,Liczba wyborców uprawnionych do głosowania,Nie wykorzystano kart do głosowania,"Liczba wyborców, którym wydano karty do głosowania",Liczba wyborców głosujących przez pełnomocnika,Liczba wyborców głosujących na podstawie zaświadczenia o prawie do głosowania,"Liczba wyborców, którym wysłano pakiety wyborcze",Liczba otrzymanych kopert zwrotnych,"Liczba kopert zwrotnych, w których nie było oświadczenia o osobistym i tajnym oddaniu głosu","Liczba kopert zwrotnych, w których oświadczenie nie było podpisane","Liczba kopert zwrotnych, w których nie było koperty na kartę do głosowania","Liczba kopert zwrotnych, w których znajdowała się niezaklejona koperta na kartę do głosowania",Liczba kopert na kartę do głosowania wrzuconych do urny,Liczba kart wyjętych z urny,W tym liczba kart wyjętych z kopert na kartę do głosowania,Liczba kart nieważnych,Liczba kart ważnych,Liczba głosów nieważnych,W tym z powodu postawienia znaku „X” obok nazwiska dwóch lub większej liczby kandydatów z różnych list,W tym z powodu niepostawienia znaku „X” obok nazwiska żadnego kandydata,"W tym z powodu postawienia znaku „X” wyłącznie obok nazwiska kandydata na liście, której rejestracja została unieważniona",Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY AKCJA ZAWIEDZIONYCH EMERYTÓW RENCISTÓW - ZPOW-601-21/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWICA - ZPOW-601-20/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SKUTECZNI PIOTRA LIROYA-MARCA - ZPOW-601-17/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW KOALICJA BEZPARTYJNI I SAMORZĄDOWCY - ZPOW-601-10/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19,TERYT gminy,Numer okręgu do Senatu
27410,ca04-d048-8fcf-7d31-cd72-0f6d-46d9-958a,NaN,19,119,zagranica,zagranica,"Novo Selo, Camp Maréshal de Lattre de Tassigny...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44
27411,1f5b-7d5a-492d-63c7-a3c7-55bf-3f78-be42,NaN,19,186,zagranica,zagranica,"Krajowa, Cercul Militar Craiova Strada Marin S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44
27412,d679-7a93-db51-e54a-0307-64a2-e807-b316,NaN,19,88,zagranica,zagranica,"At-Taji, Camp Taji Joint Operations Base in Ta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44
27413,40e4-6a02-9463-2c20-7a36-6b1d-a418-d65f,NaN,19,87,zagranica,zagranica,"Irbil, Agencja Konsularna RP Junction of Texas...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44
27414,61ce-8677-8823-f077-17c6-586e-cfc1-8e5d,NaN,19,1,zagranica,zagranica,Airfield Building 13982 Disney&California Loop...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44


# Obliczenie wyników wyborów komitetów w okręgach wyborczych do senatu

In [19]:
columns = ["Numer okręgu do Senatu",
           "Liczba głosów nieważnych",
           "Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów",
           "KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19",
           "KOMITET WYBORCZY AKCJA ZAWIEDZIONYCH EMERYTÓW RENCISTÓW - ZPOW-601-21/19",
           "KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19",
           "KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19",
           "KOMITET WYBORCZY PRAWICA - ZPOW-601-20/19",
           "KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19",
           "KOMITET WYBORCZY SKUTECZNI PIOTRA LIROYA-MARCA - ZPOW-601-17/19",
           "KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19",
           "KOMITET WYBORCZY WYBORCÓW KOALICJA BEZPARTYJNI I SAMORZĄDOWCY - ZPOW-601-10/19",
           "KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19"]

In [20]:
sejm_voting_results_2019_grouped = sejm_voting_results_2019_new[columns].groupby("Numer okręgu do Senatu").sum().astype(int)

In [21]:
sejm_voting_results_2019_grouped

,Liczba głosów nieważnych,Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY AKCJA ZAWIEDZIONYCH EMERYTÓW RENCISTÓW - ZPOW-601-21/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWICA - ZPOW-601-20/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SKUTECZNI PIOTRA LIROYA-MARCA - ZPOW-601-17/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW KOALICJA BEZPARTYJNI I SAMORZĄDOWCY - ZPOW-601-10/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19
Numer okręgu do Senatu,,,,,,,,,,,,
1,1878,118333,26757,0,7432,11123,0,48743,0,20050,4228,0
2,1697,127376,38369,0,6832,8996,0,47846,0,23256,2077,0
3,2099,186727,43065,0,11055,10887,0,86775,0,27755,7190,0
4,1851,141842,50522,0,7700,6947,0,53917,0,19348,3408,0
5,2125,141160,40290,0,7646,13581,0,60811,0,15609,3223,0
...,...,...,...,...,...,...,...,...,...,...,...,...
96,2076,146956,31521,0,8649,19369,0,66213,0,21204,0,0
97,1840,233756,95046,0,16085,10992,0,74257,0,37376,0,0
98,3948,236773,72976,0,14659,23815,0,90943,0,34380,0,0


# Przeliczenie na liczbę mandatów

## Próg wyborczy

Wybór komitetów które przekroczyły 5% w skali całego kraju

In [22]:
comitees = sejm_voting_results_2019_grouped.sum() / sejm_voting_results_2019_grouped["Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów"].sum()
comitees = comitees[comitees > 0.05]
comitees = comitees.index.values.tolist()
comitees

['Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów',
 'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19',
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19',
 'KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19',
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19',
 'KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19']

Dodanie Mniejszości Niemieckiej (nie obowiązuje jej próg wyborczy)

In [23]:
comitees.append("KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19")

In [24]:
comitees

['Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów',
 'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19',
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19',
 'KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19',
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19',
 'KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19',
 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19']

## Dodanie do wyników informacji o liczbie posłów w każdym z okręgów

In [28]:
columns = ["Numer okręgu do Senatu", "Liczba mandatów"] + comitees

sejm_voting_results_2019_trimmed = pd.merge(sejm_voting_results_2019_grouped.reset_index(), 
                                            voting_districts_grouped[["Numer okręgu do Senatu", "Liczba mandatów"]],
                                            on = "Numer okręgu do Senatu")[columns]                     

In [29]:
sejm_voting_results_2019_trimmed

,Numer okręgu do Senatu,Liczba mandatów,Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19
0,1,3,118333,26757,7432,11123,48743,20050,0
1,2,3,127376,38369,6832,8996,47846,23256,0
2,3,5,186727,43065,11055,10887,86775,27755,0
3,4,4,141842,50522,7700,6947,53917,19348,0
4,5,4,141160,40290,7646,13581,60811,15609,0
...,...,...,...,...,...,...,...,...,...
95,96,4,146956,31521,8649,19369,66213,21204,0
96,97,5,233756,95046,16085,10992,74257,37376,0
97,98,7,236773,72976,14659,23815,90943,34380,0
98,99,4,137098,43256,7692,13160,52085,20905,0


## Obliczenie Liczby Posłów

Implementacja metody d'Hondta - na poziomie pojedynczego okręgu:

https://pl.wikipedia.org/wiki/Metoda_D’Hondta

Jako wejście przyjmuje ona liczbę mandatów do obsadzenia oraz wyniki kolejnych komitetów

1. Utworzenie DataFrame'u zawierającego: 
  * "numer" komitetu, 
  * numer ilorazu d'Hondta dla danego komitetu (to przez jaką liczbę całkowitą została podzielona liczba głosów) - dla każdego komitetu wyznaczane jest tyle ilorazów jaka jest maksymalna liczba mandatów do obsadzenia,
  * iloraz d'Hondta (liczba głosów podzielona przez kolejną liczbę całkowitą), 
  * liczbę głosów
2. Uszeregowanie ilorazów d'Hondta od największego. W razie remisu brany jest komitet z całkowitą większą liczbą głosów
3. Wybór największych ilorazów
4. Zsumowanie rezultatów dla każdego z komitetów

In [51]:
def d_hondt_method(mandates: int, *args) -> tuple:
    parties_df = [(counter, n, votes_number/(n+1), votes_number) for counter, votes_number in enumerate(args) for n in range(mandates)]
    parties_df = pd.DataFrame(parties_df)
    parties_df.columns=["party_number", "quotient_number", "quotient", "votes_number"]
    parties_df = parties_df.sort_values(by=["quotient", "votes_number"], ascending = False)\
                           .reset_index(drop = True)\
                           .reset_index()\
                           .rename(columns={"index": "order"})
    parties_df["mandate"] = np.where(parties_df['order']<mandates, 1, 0)
    results = parties_df.groupby("party_number")['mandate'].sum()

    return tuple(results.to_list())

Obliczenie dla wszystkich okręgów z wykorzystaniem `np_vectorize()`

In [52]:
(sejm_voting_results_2019_trimmed["Liczba posłów - KO"],
 sejm_voting_results_2019_trimmed["Liczba posłów - Konfederacja"],
 sejm_voting_results_2019_trimmed["Liczba posłów - PSL"],
 sejm_voting_results_2019_trimmed["Liczba posłów - PiS"],
 sejm_voting_results_2019_trimmed["Liczba posłów - Lewica"],
 sejm_voting_results_2019_trimmed["Liczba posłów - MN"]) = np.vectorize(d_hondt_method)\
    (sejm_voting_results_2019_trimmed['Liczba mandatów'],
     sejm_voting_results_2019_trimmed['KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19'],
     sejm_voting_results_2019_trimmed['KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19'],
     sejm_voting_results_2019_trimmed['KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19'],
     sejm_voting_results_2019_trimmed['KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19'],
     sejm_voting_results_2019_trimmed['KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19'],
     sejm_voting_results_2019_trimmed['KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19'])

In [53]:
sejm_voting_results_2019_trimmed

,Numer okręgu do Senatu,Liczba mandatów,Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19,Liczba posłów - KO,Liczba posłów - Konfederacja,Liczba posłów - PSL,Liczba posłów - PiS,Liczba posłów - Lewica,Liczba posłów - MN
0,1,3,118333,26757,7432,11123,48743,20050,0,1,0,0,2,0,0
1,2,3,127376,38369,6832,8996,47846,23256,0,1,0,0,2,0,0
2,3,5,186727,43065,11055,10887,86775,27755,0,1,0,0,3,1,0
3,4,4,141842,50522,7700,6947,53917,19348,0,2,0,0,2,0,0
4,5,4,141160,40290,7646,13581,60811,15609,0,1,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,4,146956,31521,8649,19369,66213,21204,0,1,0,0,3,0,0
96,97,5,233756,95046,16085,10992,74257,37376,0,2,0,0,2,1,0
97,98,7,236773,72976,14659,23815,90943,34380,0,3,0,0,3,1,0
98,99,4,137098,43256,7692,13160,52085,20905,0,2,0,0,2,0,0


Wyniki całościowe:

In [67]:
sejm_voting_results_2019_condensed = sejm_voting_results_2019_trimmed.sum().to_frame().T
sejm_voting_results_2019_condensed = sejm_voting_results_2019_condensed.drop(["Numer okręgu do Senatu"], axis=1)
sejm_voting_results_2019_condensed

,Liczba mandatów,Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19,Liczba posłów - KO,Liczba posłów - Konfederacja,Liczba posłów - PSL,Liczba posłów - PiS,Liczba posłów - Lewica,Liczba posłów - MN
0,460,18469909,5060246,1256880,1578471,8051449,2319865,32094,144,0,10,276,29,1


# Zapisanie rezultatów

Dodanie informacji o zasięgu geograficznym:

In [70]:
sejm_voting_results_2019_combined = pd.merge(senat_districts_teritory,
                                             sejm_voting_results_2019_trimmed,
                                             left_on = "Numer okręgu",
                                             right_on = "Numer okręgu do Senatu")\
                                    .drop(["Numer okręgu do Senatu"], axis = 1)
sejm_voting_results_2019_combined

,Siedziba OKW,Numer okręgu,Liczba kandydatów,Mieszkańcy,Wyborcy,Opis granic,Liczba mandatów,Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19,Liczba posłów - KO,Liczba posłów - Konfederacja,Liczba posłów - PSL,Liczba posłów - PiS,Liczba posłów - Lewica,Liczba posłów - MN
0,Legnica,1,3,268968,219010,"obszary powiatów: bolesławiecki, lubański, lwó...",3,118333,26757,7432,11123,48743,20050,0,1,0,0,2,0,0
1,Legnica,2,3,266939,219472,"obejmujący obszary powiatów: jaworski, jelenio...",3,127376,38369,6832,8996,47846,23256,0,1,0,0,2,0,0
2,Legnica,3,3,391543,313689,"obszary powiatów: głogowski, legnicki, lubińsk...",5,186727,43065,11055,10887,86775,27755,0,1,0,0,3,1,0
3,Wałbrzych,4,3,303901,250787,"obszary powiatów: świdnicki, wałbrzyski oraz m...",4,141842,50522,7700,6947,53917,19348,0,2,0,0,2,0,0
4,Wałbrzych,5,3,312480,258392,"obszary powiatów: dzierżoniowski, kłodzki, ząb...",4,141160,40290,7646,13581,60811,15609,0,1,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Kalisz,96,2,309497,248880,"obszary powiatów: jarociński, kaliski, pleszew...",4,146956,31521,8649,19369,66213,21204,0,1,0,0,3,0,0
96,Szczecin,97,2,433155,352634,obszar powiatu: policki oraz miasta na prawach...,5,233756,95046,16085,10992,74257,37376,0,2,0,0,2,1,0
97,Szczecin,98,2,548117,444089,"obszary powiatów: goleniowski, gryficki, gryfi...",7,236773,72976,14659,23815,90943,34380,0,3,0,0,3,1,0
98,Koszalin,99,3,318648,258025,"obszary powiatów: białogardzki, choszczeński, ...",4,137098,43256,7692,13160,52085,20905,0,2,0,0,2,0,0


In [71]:
with pd.ExcelWriter('Wyniki wyborów w okręgach senackich.xlsx') as writer:  
    sejm_voting_results_2019_combined.to_excel(writer, sheet_name='Wyniki w okręgach', index = False)
    sejm_voting_results_2019_condensed.to_excel(writer, sheet_name='Wyniki całościowe', index = False)